In [1]:
import pandas as pd
import sqlite3

In [2]:

try:
    sqlite_connection = sqlite3.connect('sqlite_python.db')
    cursor = sqlite_connection.cursor()
    print("База данных создана и успешно подключена к SQLite")

    sqlite_select_query = "select sqlite_version();"
    cursor.execute(sqlite_select_query)
    record = cursor.fetchall()
    print("Версия базы данных SQLite: ", record)
    cursor.close()

except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)


База данных создана и успешно подключена к SQLite
Версия базы данных SQLite:  [('3.35.4',)]


In [3]:
def extract_currency(date):
    
    url = f'https://api.exchangerate.host/timeseries?start_date={date}&end_date={date}&base=EUR&symbols=USD&format=csv'
    data = pd.read_csv(url)
    return data

In [4]:
# Напишите функцию которая скачивает данные с гитхаба
# Функция принимает на вход дату
# Функция должна вернуть список словарей или можно использовать можно использоват pandas.read_csv()

# ссылка на гитхаб с файлами - https://github.com/dm-novikov/stepik_airflow_course/tree/main/data_new\

date='2021-01-04'

def extract_data(date):
    url = f'https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/{date}.csv'
    data = pd.read_csv(url)
    return data


In [5]:
# Напишите функцию которая принимает на вход список словарей
# и загружает в табличку в sqlite, можно использоват pandas.to_sql()

def insert_to_db(data, table_name):
    data.to_sql(table_name, sqlite_connection, if_exists='replace', index=False)

In [6]:
# Напишите функцию которая выполнит SQL запрос к базу данных
# Здесь удобно использовать sqlalchemy или https://pythonru.com/biblioteki/vvedenie-v-sqlite-python
# Если выполняется select ... то функция должна возвращать результат (не None)
# Если используется внутренняя операция например truncate table, insert into select ... то возвращает None
# data = sql_query(sql="select * from table") -> [(1, 2, 3), (4, 5, 6) ...]
# sql_query("truncate table") -> None
def sql_query(sql):
  cursor = sqlite_connection.cursor()
  cursor.execute(sql)
  cursor.close()


In [7]:
# Запустите ваш код в функции main

# Напишите генерацию дат, так чтобы у вас получился список
# 2021-01-02, 2021-01-03 ... etc
# Нужны даты с 2021-01-01 по 2021-01-04

dates_list = ['2021-01-04']

def main(date):
  
  # Выгружаем данные по валютам и из источника
  currency = extract_currency(date)
  data = extract_data(date)

  # Создайте необходимые таблицы если это нужно
  # 'currency', 'data' и 'join_data (таблица с объединенными данными)
  # sql_query() 
  # ...
  # ...
  # ...

  # Вставляем данные в БД
  # У вас долнжо получиться 2 таблицы CURRENCY, DATA
  insert_to_db(currency, 'currency')
  insert_to_db(data, 'data')

  # Объединение данных в отдельную таблицу JOIN_DATA
  # CURRENCY, DATA объединить через JOIN по дате и валюте
  # Используйте insert into select ... inner join ...
  # И соответственно создать таблицу для этого
  sql_query('create table JOIN_DATA as select a.*, b.* from CURRENCY a inner join data b on a.date=b.date')

  # Очистка временных таблиц CURRENCY, DATA
  # Это те таблицы куда произошла вставка данных
  # Используйте truncate
  # sql_query()

  # Получение данных из таблицы JOIN_DATA за конкретный день
  # Используйте date при формировании запроса select * from ... where date ='...'
  # report = sql_query()

# Пройдемся по списку дат и выполним скрипт
for date in dates_list:
  main(date)

In [9]:
try:
    cursor = sqlite_connection.cursor()
    sqlite_select_query = "select * from JOIN_DATA;"
    cursor.execute(sqlite_select_query)
    record = cursor.fetchall()
    print( record)
    cursor.close()

except sqlite3.Error as error:
    print("Ошибка при подключении к sqlite", error)

[('2021-01-04', 'USD', '1,225061', 'EUR', '2021-01-04', '2021-01-04', 'EUR', 20, '2021-01-04'), ('2021-01-04', 'USD', '1,225061', 'EUR', '2021-01-04', '2021-01-04', 'EUR', 33, '2021-01-04'), ('2021-01-04', 'USD', '1,225061', 'EUR', '2021-01-04', '2021-01-04', 'EUR', 38, '2021-01-04'), ('2021-01-04', 'USD', '1,225061', 'EUR', '2021-01-04', '2021-01-04', 'EUR', 39, '2021-01-04'), ('2021-01-04', 'USD', '1,225061', 'EUR', '2021-01-04', '2021-01-04', 'EUR', 51, '2021-01-04'), ('2021-01-04', 'USD', '1,225061', 'EUR', '2021-01-04', '2021-01-04', 'EUR', 57, '2021-01-04'), ('2021-01-04', 'USD', '1,225061', 'EUR', '2021-01-04', '2021-01-04', 'EUR', 68, '2021-01-04'), ('2021-01-04', 'USD', '1,225061', 'EUR', '2021-01-04', '2021-01-04', 'EUR', 88, '2021-01-04'), ('2021-01-04', 'USD', '1,225061', 'EUR', '2021-01-04', '2021-01-04', 'EUR', 90, '2021-01-04')]
